# Sales Data Engineering Pipeline (SQL-Ready)

## Purpose
This notebook prepares raw sales data into clean, validated, and normalized tables
ready for SQL modeling and Power BI consumption.

**Python is used strictly for data engineering and validation not analytics.**

## Outputs
- fact_sales
- dim_product
- dim_city
- dim_date


## 1. Data Ingestion & Profiling

Before any transformation, we profile the dataset to understand:
- Row counts
- Column roles
- Data grain
- Structural issues

This prevents blind cleaning and ensures business logic is respected.
## (check data_quality_report for more)


In [32]:
#import libraries
import pandas as pd
import numpy as np
import hashlib


In [33]:
#load 
df = pd.read_csv("../data/raw/sales_data.csv")


In [34]:
df.head()


,Unnamed: 0,Order ID,Product Category,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Hour,Time of Day
0,0,295665,Laptops and Computers,Macbook Pro Laptop,1,1700.00,30-12-2019 00:01,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0,Night
1,1,295666,Home Appliances,LG Washing Machine,1,600.00,29-12-2019 07:03,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7,Morning
2,2,295667,Charging Cables,USB-C Charging Cable,1,11.95,12-12-2019 18:21,"277 Main St, New York City, NY 10001",12,11.95,New York City,18,Evening
3,3,295668,Monitors,27in FHD Monitor,1,149.99,22-12-2019 15:13,"410 6th St, San Francisco, CA 94016",12,149.99,San Francisco,15,Afternoon
4,4,295669,Charging Cables,USB-C Charging Cable,1,11.95,18-12-2019 12:38,"43 Hill St, Atlanta, GA 30301",12,11.95,Atlanta,12,Afternoon


In [35]:
df.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
185945    False
185946    False
185947    False
185948    False
185949    False
Length: 185950, dtype: bool

# Type Enforcement & Canonical Time Fields

We explicitly enforce data types and create canonical datetime fields
to simplify SQL joins, time intelligence, and Power BI measures.


In [36]:
# Strip text fields
text_cols = ["Order ID", "Product", "Purchase Address"]
for col in text_cols:
    if col in df.columns and pd.api.types.is_string_dtype(df[col]):
        df[col] = df[col].str.strip()

# Type enforcement
df["Quantity Ordered"] = pd.to_numeric(df["Quantity Ordered"], errors="coerce")
df["Price Each"] = pd.to_numeric(df["Price Each"], errors="coerce")

# Datetime handling
df["order_datetime"] = pd.to_datetime(df["Order Date"], errors="coerce")
df["order_date"] = df["order_datetime"].dt.date
df["order_year"] = df["order_datetime"].dt.year
df["order_month"] = df["order_datetime"].dt.month
df["order_hour"] = df["order_datetime"].dt.hour


C:\Users\Envy\AppData\Local\Temp\ipykernel_17992\4066972160.py:12: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_datetime"] = pd.to_datetime(df["Order Date"], errors="coerce")


In [37]:
df.columns


Index(['Unnamed: 0', 'Order ID', 'Product Category', 'Product',
       'Quantity Ordered', 'Price Each', 'Order Date', 'Purchase Address',
       'Month', 'Sales', 'City', 'Hour', 'Time of Day', 'order_datetime',
       'order_date', 'order_year', 'order_month', 'order_hour'],
      dtype='object')

Business Rule Validation

Instead of deleting data, we **flag invalid records**.
This preserves traceability and supports auditability.

In [38]:
df["calculated_sales"] = df["Quantity Ordered"] * df["Price Each"]

df["is_valid_record"] = (
    (df["Quantity Ordered"] > 0) &
    (df["Price Each"] > 0) &
    (np.isclose(df["calculated_sales"], df["Sales"], atol=0.01))
)


##  Feature Engineering (SQL-Enabling)

These fields reduce SQL complexity and eliminate the need
for Power BI DAX hacks.


In [39]:
# Time of day
def time_of_day(hour):
    if hour < 6:
        return "Night"
    elif hour < 12:
        return "Morning"
    elif hour < 18:
        return "Afternoon"
    else:
        return "Evening"

df["time_of_day"] = df["order_hour"].apply(time_of_day)

# Price band
df["price_band"] = pd.cut(
    df["Price Each"],
    bins=[0, 50, 150, np.inf],
    labels=["Low", "Medium", "High"]
)

# Product SKU
df["product_sku"] = df["Product"].apply(
    lambda x: hashlib.md5(x.encode()).hexdigest()[:10]
)

# City extraction
df["city"] = df["Purchase Address"].str.split(",").str[1].str.strip()



 Normalized Table Preparation

Python prepares dimension and fact tables.
SQL should **load models**, not derive them.


In [40]:
# dim_product
dim_product = (
    df[["product_sku", "Product", "price_band"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_product.columns = ["product_id", "product", "price_band"]

# dim_city
dim_city = (
    df[["city"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_city["city_id"] = dim_city.index + 1

# dim_date
dim_date = (
    df[["order_date", "order_year", "order_month", "order_hour", "time_of_day"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_date["date_id"] = dim_date.index + 1


In [41]:
fact_sales = df.merge(dim_product, left_on="product_sku", right_on="product_id") \
               .merge(dim_city, on="city") \
               .merge(dim_date, on=["order_date", "order_year", "order_month", "order_hour", "time_of_day"])

fact_sales = fact_sales[[
    "Order ID",
    "product_id",
    "date_id",
    "city_id",
    "Quantity Ordered",
    "Price Each",
    "Sales",
    "is_valid_record"
]]

fact_sales.columns = [
    "order_id",
    "product_id",
    "date_id",
    "city_id",
    "quantity",
    "price",
    "sales",
    "is_valid_record"
]


 Export SQL-Ready Tables

Only normalized, validated tables are exported.
Raw data never enters BI tools.


In [42]:
dim_product.to_csv("../data/processed/dim_product.csv", index=False)
dim_city.to_csv("../data/processed/dim_city.csv", index=False)
dim_date.to_csv("../data/processed/dim_date.csv", index=False)
fact_sales.to_csv("../data/processed/fact_sales.csv", index=False)
